In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
#계산 그래프 만들기

x = tf.Variable(3, name='x') #tf에 속한 변수를 선언해주고, 변수에 값을 넣고, 이름을 지정해주는 순서
y = tf.Variable(4, name='y')

f = x*x*y +y + 2

## 사람으로 생각하면, 먼저 식을 세우고, sesstion을 새로 열어주어 계산을 하는 방식

# Session

In [3]:
# 방법 1

sess = tf.Session() #세션을 선언
sess.run(x.initializer) 
sess.run(y.initializer)
# 계산하기 위해서 run 

result = sess.run(f) #f를 계산한걸 Result에 담았고,
print(result) # 그 결과값을 보여줄 것
sess.close() #세션을 닫아라

42


## with 함수

기본적으로 try finally 구문이라는게 있는데,

try를 선언할 때 특정 조건을 써주고, 이 조건을 충족할 때까지 try 내부의 구문이 계속 돌아가다가 try의 구문의 조건을 충족시키면 try구문을 종료하고 finally 문구를 실행한 후 종료되는 구문

위 방식을 간단하게 with 함수로 해결할 수 있음

try finally 함수와 with 함수 : https://wikidocs.net/16078

In [4]:
# 방법 2 with 함수를 사용하는 방법, 일일이 변수들을 초기화 해줘야하는 단점이 있음

with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run() 
    result = f.eval()
print(result)


42


In [5]:
# 방법 2.1 initializer를 전역변수로 설정해 모든 변수를 초기화하는 방법

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

# 계산 그래프 관리

기본적으로 텐서플로는 계산그래프 + 실행의 구조로 되어있음.

코드짤 때, 계산그래프 클래스, (실행,평가)클래스를 따로 짠다면 효율적일 것 같음

In [6]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [7]:
# 여러개 계산 그래프가 필요할 때

graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)


In [8]:
x2.graph is graph #계산 그래프가 여러개일 때는 tf.Graph()를 사용하도록 하자!

True

In [9]:
x2.graph is tf.get_default_graph() #get_default_graph()를 사용하면 false가 나오는 것을 알 수 있음

False

In [10]:
#계산 그래프 실행시킬 때 주의해야할 점
#잘못된 예

w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess :
    print(y.eval()) 
    print(z.eval())

10
15


위와 같은 경우 y를 계산하기 위해 x를 구하고, w를 구함. 그리고 다시 z를 구하기 위해 x와 w를 구하는 비효율적인 상황이 발생함

따라서 eval할 때, array 형식으로 선언해줘야함

In [11]:
#수정

with tf.Session() as sess :
    y_val, z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


## 정규방정식을 이용해서 선형회귀 분석의 가중치 구하기

In [22]:
import numpy as np
from sklearn.datasets import fetch_california_housing
import sklearn

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = 'X')
y = tf.constant(housing.target.reshape(-1,1), dtype =tf.float32, name = 'y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
    theta_values = theta.eval()
    print(theta_values)

[[-3.6894890e+01]
 [ 4.3661433e-01]
 [ 9.4453208e-03]
 [-1.0704148e-01]
 [ 6.4345831e-01]
 [-3.9632569e-06]
 [-3.7880042e-03]
 [-4.2093179e-01]
 [-4.3400639e-01]]


## 경사하강법을 이용한 최적값 찾기

In [46]:
#방법1 : 직접 기울기를 구함
n_epochs = 1000
learning_rate = 0.01
scaled_housing_data_plus_bias = housing_data_plus_bias/(housing_data_plus_bias.max()-housing_data_plus_bias.min())
#데이터를 정규화 시키지 않으면 시간이 매우 오래 걸림.

#변수 선언 
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,-1.0), dtype=tf.float32, name ='theta') 
#n ==> 위에 있는 data col 개수
y_pred = tf.matmul(X,theta, name='predictions') # 간단한 선형회귀식 선언
error = y_pred - y #에러값을 구해주고
mse = tf.reduce_mean(tf.square(error), name='mse') #에러 제곱항의 평균을 구하는 식 cost function으로 mse를 사용한 것을 알 수 있음
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta-learning_rate * gradients) # assign은 갱신 (:=) 

init = tf.global_variables_initializer()

with tf.Session() as sess :
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE = ',mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE =  5.7713094
Epoch 100 MSE =  5.74301
Epoch 200 MSE =  5.715004
Epoch 300 MSE =  5.687286
Epoch 400 MSE =  5.6598525
Epoch 500 MSE =  5.6327033
Epoch 600 MSE =  5.605834
Epoch 700 MSE =  5.5792403
Epoch 800 MSE =  5.552922
Epoch 900 MSE =  5.5268745


In [14]:
# 방법2 옵티마이저 이용 : 실제로 사용하는 방법
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)

## feed dict을 이용해 계산그래프에 데이터 넣기

In [15]:
#placeholder를 만들어서, 데이터가 들어갈 차원을 입력해두자.

A = tf.placeholder(tf.float32, shape=(None,3)) 
#none은 데이터 차원을 제한하지 않는거고, 예제에서는 데이터를 3개씩 넣어서 [none,3]으로 지정
B = A+5

with tf.Session() as sess :
    B_val_1 = B.eval(feed_dict={A:[[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A: [[4,5,6],[7,8,9]]})

print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


# 경사하강법 구현 미니배치사용

In [33]:
import numpy as np
from sklearn.datasets import fetch_california_housing
import sklearn

In [34]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data] #m,1짜리 1로만 이루어진 행렬(vias)를 housing data에 concat시킨것

n_epochs = 1000
learning_rate = 0.01

scaled_housing_data_plus_bias = housing_data_plus_bias/(housing_data_plus_bias.max()-housing_data_plus_bias.min())
#정규화

In [35]:
tf.reset_default_graph()


X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

batch_size = 100
n_batches = int(np.ceil(m/batch_size))

In [36]:
def fetch_batch(epoch, batch_index, batch_size):
    
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices]
    
    return X_batch, y_batch

In [40]:
#모델 저장

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess :
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0 :
            save_path = saver.save(sess,'/Users/chaehuiung/Desktop/project/work/mnist/model/linear_regression/my_model.ckpt')
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict = {X:X_batch, y:y_batch})
            
        best_theta = theta.eval()
        save_path = saver.save(sess,'/Users/chaehuiung/Desktop/project/work/mnist/model/linear_regression/my_model.ckpt')
        


In [41]:
#저장된 모델 사용

with tf.Session() as sess:
    saver.restore(sess, '/Users/chaehuiung/Desktop/project/work/mnist/model/linear_regression/my_model.ckpt')
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /Users/chaehuiung/Desktop/project/work/mnist/model/linear_regression/my_model.ckpt


In [45]:
best_theta_restored

array([[  1.0107152],
       [  0.9394269],
       [  4.1914196],
       [  1.1565114],
       [  0.0686223],
       [ 30.712315 ],
       [ -0.4105714],
       [  2.9946258],
       [-13.0559845]], dtype=float32)

In [47]:
# 실습 moon dataset으로 미니배치 경사하강법으로 logistic regression 구현해보기
# 방법1 tensorflow

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=100, noise=0.15, random_state=42)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123456) #feed_dic하면 되는 애들

In [64]:
import tensorflow as tf

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
y_proba = tf.sigmoid(y_pred)

error = y * tf.log(y_proba) + (1-y) * tf.log(1-y_proba)
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

batch_size = 100
n_batches = int(np.ceil(m/batch_size))

In [65]:
#모델 저장

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess :
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0 :
            save_path = saver.save(sess,'/Users/chaehuiung/Desktop/project/work/mnist/model/linear_regression/my_model.ckpt')
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict = {X:X_batch, y:y_batch})
            
        best_thet0a = theta.eval()
        save_path = saver.save(sess,'/Users/chaehuiung/Desktop/project/work/mnist/model/linear_regression/my_model.ckpt')
        

